In [1]:
# カレントディレクトリをリポジトリ直下にするおまじない
import os
import tensorflow as tf 
print(tf.__version__)
print("hello world")
print(os.getcwd().split('\\')[-1] )
while os.getcwd().split('\\')[-1] != 'deep_dialog_tutorial': os.chdir('..')
print('current dir:', os.getcwd())


AttributeError: module 'tensorflow' has no attribute '__version__'

In [2]:
import os
import tensorflow as tf
from deepdialog.transformer.transformer import Transformer
from deepdialog.transformer.preprocess.batch_generator import BatchGenerator

AttributeError: module 'tensorflow.compat.v2.__internal__.tracking' has no attribute 'DelegatingTrackableMixin'

# Create Data

In [ ]:
data_path = 'data\\pro-jpn3.txt'

In [ ]:
batch_generator = BatchGenerator()
batch_generator.load(data_path)

In [ ]:
vocab_size = batch_generator.vocab_size

# Create Model

In [ ]:
graph = tf.Graph()
with graph.as_default():
    transformer = Transformer(
        vocab_size=vocab_size,
        hopping_num=4,
        head_num=8,
        hidden_dim=512,
        dropout_rate=0.1,
        max_length=50,
    )
    transformer.build_graph()

# Create Training Graph

In [ ]:
save_dir = 'tmp/learning/transformer/'
log_dir = os.path.join(save_dir, 'log')
ckpt_path = os.path.join(save_dir, 'checkpoints/model.ckpt')

os.makedirs(log_dir, exist_ok=True)

In [ ]:
with graph.as_default():
    global_step = tf.compat.v1.train.get_or_create_global_step()
    
    learning_rate = tf.compat.v1.placeholder(dtype=tf.float32, name='learning_rate')
    optimizer = tf.compat.v1.train.AdamOptimizer(
        learning_rate=learning_rate,
        beta2=0.98,
    )
    #TensorFlowには損失関数を最小限に抑えるために各変数をゆっくりと変更するオプティマイザが用意されている
    optimize_op = optimizer.minimize(transformer.loss, global_step=global_step)
# Tensor boardに出すためにデータをマージ
    summary_op = tf.compat.v1.summary.merge([
        tf.compat.v1.summary.scalar('train/loss', transformer.loss),
        tf.compat.v1.summary.scalar('train/acc', transformer.acc),
        tf.compat.v1.summary.scalar('train/learning_rate', learning_rate),
    ], name='train_summary')
    summary_writer = tf.compat.v1.summary.FileWriter(log_dir, graph)
    saver = tf.compat.v1.train.Saver()

# Train

In [ ]:
max_step = 10000
batch_size = 128
max_learning_rate = 0.0001
warmup_step = 4000

In [ ]:
def get_learning_rate(step: int) -> float:
    rate = min(step ** -0.5, step * warmup_step ** -1.5) / warmup_step ** -0.5
    return max_learning_rate * rate

In [ ]:
with graph.as_default():
    sess = tf.compat.v1.Session()
    sess.run(tf.compat.v1.global_variables_initializer())
    step = 0

In [ ]:
with graph.as_default():
    for batch in batch_generator.get_batch(batch_size=batch_size):
        feed = {
            **batch,
            learning_rate: get_learning_rate(step + 1),
        }
        print(np.shape(batch))
       # _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
        summary_writer.add_summary(summary, step)
        
        if step % 100 == 0:
            print(f'{step}: loss: {loss},\t acc: {acc}')
            saver.save(sess, ckpt_path, global_step=step)
        if step == max_step:
            break

In [ ]:
with graph.as_default():
    saver.save(sess,'transformer.model')

In [ ]:
test = 'GameObject obj = this.gameObject;'
test = batch_generator._create_question(test)
# test = tf.convert_to_tensor(test)

result = transformer.predict(test)


In [ ]:
self.is_training = tf.compat.v1.placeholder(dtype=tf.bool, name='is_training')
# [batch_size, max_length]
self.encoder_input = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, None], name='encoder_input')
# [batch_size]
self.decoder_input = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, None], name='decoder_input')
logit = self.call(
                encoder_input=self.encoder_input,
                decoder_input=self.decoder_input[:, :-1],  # 入力は EOS を含めない
                training=self.is_training,
            )
with Session() as sess:
    translate = tf.nn.softmax(logit, name='prediction')

In [ ]:
for batch in batch_generator.get_batch(batch_size=batch_size):
    print(batch.shape)
    break

In [ ]:
test = 'GameObject obj = this.gameObject;'
test = batch_generator._create_question(test)
test = batch_generator._convert_to_array(test)
with graph.as_default():
    # for batch in batch_generator.get_batch(batch_size=batch_size):
    feed = {
        **test,
        learning_rate: get_learning_rate(step + 1),
    }
    _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
    